<a href="https://colab.research.google.com/github/geoskimoto/SNOTEL_AnomalyDetection/blob/main/AnomalyDetection_usinglocalCSV_6_27_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To get this ready for analysis:

*   Create additional logical tests: 
    * Scatter plot of SWE vs. Precip (should be line with slope of 1, or at least never changing)
    * Plot for precip (never   should be <0).  Can probabjust use dmp for this check.
    * Plot the incremental SWE, SNWD, and PREC and look for outliers/very large changes
    *SWE decreasing when temp > 0 deg C


*   Add single line double mass plot

Longer term:

Convert all these functions into a class(es) and clean up organization and code





# Import and Data Munging

### Library imports

In [93]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,15)

In [2]:
try:
    import pyod
    print("module 'pyod' is installed")
except ModuleNotFoundError:
  !pip install pyod

module 'pyod' is installed


In [3]:
try:
    import hampel
    print("module 'hampel' is installed")
except ModuleNotFoundError:
  !pip install hampel

module 'hampel' is installed


In [35]:
!pip install statsmodels==0.12.1

     |████████████████████████████████| 9.5MB 4.9MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [6]:
# from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
# from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
from scipy.stats import zscore 
# import statsmodels.api as sm
from functools import reduce
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
import statsmodels.api as sm

import datetime

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from pyod.models.lof import LOF
from sklearn.neighbors import LocalOutlierFactor
import hampel
import statsmodels.tsa.filters as filters
from statsmodels.tsa.filters.bk_filter import bkfilter

from scipy.stats import zscore 

In [7]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import scipy.stats as stats
# import statsmodels.api as sm
# from functools import reduce

### Data Import and Prep

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
cd /content/gdrive/My Drive/Projects/Work/AnomalyDetection/Data

/content/gdrive/My Drive/Projects/Work/AnomalyDetection/Data


In [12]:
ls

all_swe                   OR_1991-2021.xlsx      WA_1991-2021_temps.xlsx
Correlated_Stations.xlsx  OR_WY1990-2020.xlsx    WA_1991-2021.xlsx
Data_parsing.ipynb        SWEvsPrec/             WA_WY1990-2020.xlsx
OR_1991-2021_raw.xlsx     TempOutliers/
OR_1991-2021_temps.xlsx   WA_1991-2001_raw.xlsx


In [13]:
WA_Sites = pd.read_excel('WA_WY1990-2020.xlsx')
# pd.to_datetime(WA_Sites['Date'])
WA_Sites.set_index('Date',inplace=True)
# WA_Sites.sample(5)

OR_Sites = pd.read_excel('OR_WY1990-2020.xlsx')
# pd.to_datetime(OR_Sites['Date'])
OR_Sites.set_index('Date',inplace=True)
# OR_Sites.sample(5)

ORWA_Sites = pd.merge(OR_Sites,WA_Sites,left_index=True,right_index=True)
ORWA_Sites.head(1)

,Aneroid Lake #2 (302) Precipitation Accumulation (in) Start of Day Values,Aneroid Lake #2 (302) Precipitation Increment (in),Aneroid Lake #2 (302) Precipitation Increment - Snow-adj (in),Aneroid Lake #2 (302) Snow Depth (in) Start of Day Values,Aneroid Lake #2 (302) Snow Water Equivalent (in) Start of Day Values,Annie Springs (1000) Precipitation Accumulation (in) Start of Day Values,Annie Springs (1000) Precipitation Increment (in),Annie Springs (1000) Precipitation Increment - Snow-adj (in),Annie Springs (1000) Snow Depth (in) Start of Day Values,Annie Springs (1000) Snow Water Equivalent (in) Start of Day Values,Anthony Lake (1245) Precipitation Accumulation (in) Start of Day Values,Anthony Lake (1245) Precipitation Increment (in),Anthony Lake (1245) Precipitation Increment - Snow-adj (in),Anthony Lake (1245) Snow Depth (in) Start of Day Values,Anthony Lake (1245) Snow Water Equivalent (in) Start of Day Values,Arbuckle Mtn (304) Precipitation Accumulation (in) Start of Day Values,Arbuckle Mtn (304) Precipitation Increment (in),Arbuckle Mtn (304) Precipitation Increment - Snow-adj (in),Arbuckle Mtn (304) Snow Depth (in) Start of Day Values,Arbuckle Mtn (304) Snow Water Equivalent (in) Start of Day Values,Bald Mtn AM OR (447) Precipitation Accumulation (in) Start of Day Values,Bald Mtn AM OR (447) Precipitation Increment (in),Bald Mtn AM OR (447) Precipitation Increment - Snow-adj (in),Bald Mtn AM OR (447) Snow Depth (in) Start of Day Values,Bald Mtn AM OR (447) Snow Water Equivalent (in) Start of Day Values,Barney Creek (1246) Precipitation Accumulation (in) Start of Day Values,Barney Creek (1246) Precipitation Increment (in),Barney Creek (1246) Precipitation Increment - Snow-adj (in),Barney Creek (1246) Snow Depth (in) Start of Day Values,Barney Creek (1246) Snow Water Equivalent (in) Start of Day Values,Bear Flat Meadow AM (1291) Precipitation Accumulation (in) Start of Day Values,Bear Flat Meadow AM (1291) Precipitation Increment (in),Bear Flat Meadow AM (1291) Precipitation Increment - Snow-adj (in),Bear Flat Meadow AM (1291) Snow Depth (in) Start of Day Values,Bear Flat Meadow AM (1291) Snow Water Equivalent (in) Start of Day Values,Bear Grass (1166) Precipitation Accumulation (in) Start of Day Values,Bear Grass (1166) Precipitation Increment (in),Bear Grass (1166) Precipitation Increment - Snow-adj (in),Bear Grass (1166) Snow Depth (in) Start of Day Values,Bear Grass (1166) Snow Water Equivalent (in) Start of Day Values,...,Tinkham Creek (899) Precipitation Accumulation (in) Start of Day Values,Tinkham Creek (899) Precipitation Increment (in),Tinkham Creek (899) Precipitation Increment - Snow-adj (in),Tinkham Creek (899) Snow Depth (in) Start of Day Values,Tinkham Creek (899) Snow Water Equivalent (in) Start of Day Values,Touchet (824) Precipitation Accumulation (in) Start of Day Values,Touchet (824) Precipitation Increment (in),Touchet (824) Precipitation Increment - Snow-adj (in),Touchet (824) Snow Depth (in) Start of Day Values,Touchet (824) Snow Water Equivalent (in) Start of Day Values,Trinity (1171) Precipitation Accumulation (in) Start of Day Values,Trinity (1171) Precipitation Increment (in),Trinity (1171) Precipitation Increment - Snow-adj (in),Trinity (1171) Snow Depth (in) Start of Day Values,Trinity (1171) Snow Water Equivalent (in) Start of Day Values,Trough (832) Precipitation Accumulation (in) Start of Day Values,Trough (832) Precipitation Increment (in),Trough (832) Precipitation Increment - Snow-adj (in),Trough (832) Snow Depth (in) Start of Day Values,Trough (832) Snow Water Equivalent (in) Start of Day Values,Upper Wheeler (841) Precipitation Accumulation (in) Start of Day Values,Upper Wheeler (841) Precipitation Increment (in),Upper Wheeler (841) Precipitation Increment - Snow-adj (in),Upper Wheeler (841) Snow Depth (in) Start of Day Values,Upper Wheeler (841) Snow Water Equivalent (in) Start of Day Values,Waterhole (974) Precipitation Accumulation (in) Start of Day Values,Waterhole (974) Precipita

In [14]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

ORWA_Precip_Acc = ORWA_Sites[ORWA_Sites.columns[::5]]
stations = [ORWA_Precip_Acc.columns[i].rsplit(' ', 7)[0] for i in range(0, len(ORWA_Precip_Acc.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Precipitation Accumulation (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Precip_Acc.columns = header

ORWA_Precip_Incr = ORWA_Sites[ORWA_Sites.columns[1::5]]
stations = [ORWA_Precip_Incr.columns[i].rsplit(' ', 3)[0] for i in range(0, len(ORWA_Precip_Incr.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Precipitation Increment (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Precip_Incr.columns = header
# ORWA_Precip_Incr.columns = stations

ORWA_Precip_Incr_sa = ORWA_Sites[ORWA_Sites.columns[2::5]]
stations = [ORWA_Precip_Incr_sa.columns[i].rsplit(' ', 5)[0] for i in range(0, len(ORWA_Precip_Incr_sa.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Precipitation Increment - Snow-adj (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_Precip_Incr_sa.columns = header

ORWA_SNWD = ORWA_Sites[ORWA_Sites.columns[3::5]]
stations = [ORWA_SNWD.columns[i].rsplit(' ', 7)[0] for i in range(0, len(ORWA_SNWD.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Snow Depth (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_SNWD.columns = header

ORWA_SWE = ORWA_Sites[ORWA_Sites.columns[4::5]]
stations = [ORWA_SWE.columns[i].rsplit(' ', 8)[0] for i in range(0, len(ORWA_SWE.columns))]  #probably not the cleanest way to extract station name, but hey it works.
iterables = [stations, ["Water Equivalent (in)"]]
header = pd.MultiIndex.from_product(iterables, names=["station", "parameter"])
ORWA_SWE.columns = header
# ORWA_SWE.columns = stations


ORWA_Sites2 = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [ORWA_Precip_Acc,ORWA_Precip_Incr,ORWA_Precip_Incr_sa,ORWA_SNWD,ORWA_SWE])
ORWA_Sites2.head(1)


station,Aneroid Lake #2 (302),Annie Springs (1000),Anthony Lake (1245),Arbuckle Mtn (304),Bald Mtn AM OR (447),Barney Creek (1246),Bear Flat Meadow AM (1291),Bear Grass (1166),Beaver Reservoir (331),Big Red Mountain (341),Big Sheep AM (1229),Bigelow Camp (343),Billie Creek Divide (344),Blazed Alder (351),Blue Mountain Spring (357),Bourne (361),Bowman Springs (362),Buckskin Lake AM (1292),Call Meadows AM (1289),Cascade Summit (388),Chemult Alternate (395),Clackamas Lake (398),Clear Lake (401),Cold Springs Camp (406),County Line (422),Cox Flat AM (1273),Crazyman Flat (1010),Daly Lake (434),Derr. (440),Diamond Lake (442),Eilertson Meadows (464),Emigrant Springs (470),Finley Corrals AM (1274),Fish Creek (477),Fish Lk. (479),Fourmile Lake (483),Gerber Reservoir (945),Gold Center (494),Government Corrals (1250),Greenpoint (504),...,Mowich (941),Mt. Tebo (1126),Muckamuck (1259),Olallie Meadows (672),Paradise (679),Park Creek Ridge (681),Pepper Creek (1104),Pigtail Peak (692),Pinto Rock (1263),Pope Ridge (699),Potato Hill (702),Quartz Peak (707),Rainy Pass (711),Rex River (911),Salmon Meadows (728),Sasse Ridge (734),Satus Pass (1231),Sawmill Ridge (1068),Sentinel Butte (1043),Sheep Canyon (748),Skate Creek (1257),Skookum Creek (912),Sourdough Gulch (985),Spencer Meadow (776),Spirit Lake (777),Spruce Springs (984),Stampede Pass (788),Stevens Pass (791),Surprise Lakes (804),Swamp Creek (975),Swift Creek (1012),Thunder Basin (817),Tinkham Creek (899),Touchet (824),Trinity (1171),Trough (832),Upper Wheeler (841),Waterhole (974),Wells Creek (909),White Pass E.S. (863)
parameter,Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),Precipitation Accumulation (in),...,Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in),Water Equivalent (in)
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-10-01,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN

# Functions

### Timeseries plotting tool

In [ ]:
def plotData(stations, parameter_of_interest):

  fig = go.Figure()
 
  for i in stations:

    fig.add_trace(go.Scatter(
      x=ORWA_Sites2.index,
      y=ORWA_Sites2[i][parameter_of_interest],
      mode='lines',
      name=i,
      hovertext = ORWA_Sites2[i][parameter_of_interest],   
  ))


  fig.update_xaxes(title_text= 'Date')
  fig.update_yaxes(title_text= parameter_of_interest)

  fig.update_layout(
   height=800,
   width=1100,
  )

  fig.show()

In [ ]:
# stations = ['Corral Pass (418)', 'Morse Lake (642)', 'Pigtail Peak (692)']

# plotData(stations, parameter_of_interest= 'Water Equivalent (in)')

### View missing data for specified station

In [ ]:
def missingData(station, parameter_of_interest):
  fig, ax = plt.subplots(figsize=(10,150))
  sns.heatmap(ORWA_Sites2[f'{station}'][[parameter_of_interest]].isnull(), cbar=False)

In [ ]:
# missingData('Morse Lake (642)', 'Precipitation Accumulation (in)')

In [ ]:
# filter = ORWA_Sites2['Morse Lake (642)'][['Precipitation Accumulation (in)']].isnull()
# ORWA_Sites2.loc[:,('Morse Lake (642)', 'Precipitation Accumulation (in)')]

# ORWA_Sites2.isin()

In [ ]:
# filter = ORWA_Sites2.loc[:,('Morse Lake (642)', 'Precipitation Accumulation (in)')] <= 0

# pd.DataFrame(ORWA_Sites2['Morse Lake (642)'][filter].index)

### Find erroneous decreases in precip accum:

In [20]:
def decreasePrecip(station):
  filter = ORWA_Sites2.loc[:,(f'{station}', 'Precipitation Increment (in)')] < 0

  return pd.DataFrame(ORWA_Sites2['Morse Lake (642)'][filter])


In [21]:
decreasePrecip('Morse Lake (642)')

parameter,Precipitation Accumulation (in),Precipitation Increment (in),Precipitation Increment - Snow-adj (in),Snow Depth (in),Water Equivalent (in)
Date,,,,,


### SWE vs. Precip

In [ ]:
def SWEvsPrecip(station):  #If you set month to a default, python sees it as a key-value argument which will break *args.

#Create dataframe with WTEQ and Precip Accum for selected station
  df = ORWA_Sites2.loc[:, (f'{station}')].loc[:, ('Water Equivalent (in)','Precipitation Accumulation (in)')]  
  df.reset_index(inplace=True)
  pd.to_datetime(df.loc[:,'Date'])

#Mask to select only data between Nov - Apr
  winter_filter = ~df.loc[:,'Date'].dt.month.between(5,10)
  df2 = df.where(winter_filter).dropna()
  # df2.set_index('Date', inplace = True)

#Add column specifying the water year (Oct 1 - Sep 30) of each record
  df2.reset_index(inplace=True)  #Have to reset the index so the dates become just a regular datetime object to get .dt.year to work.
  pd.to_datetime(df2.loc[:,'Date'])
  df2.loc[:,'water_year'] = df2.loc[:,'Date'].dt.year.where(df2.loc[:,'Date'].dt.month < 10, df2.loc[:,'Date'].dt.year + 1)
  df2.loc[:,'water_year'] = list(map(lambda x: str(x), df2.loc[:,'water_year']))

#Line plot
  fig = px.line(df2, x="Precipitation Accumulation (in)",hover_name=df2.loc[:,'Date'], y="Water Equivalent (in)", color='water_year')

  fig.update_layout(
      yaxis=dict(scaleanchor="x", scaleratio=1),
      height=800,
      width=1100)

  fig.show()

### Hampel Filter

In [18]:
def hampel_filter(station, parameter, window_size=10 ,n=3):
  global df
  #Create dataframe with WTEQ and Precip Accum for selected station
  df = ORWA_Sites2.loc[:, (f'{station}')].loc[:, (f'{parameter}')]  
  # df.reset_index(inplace=True)
  # pd.to_datetime(df.loc[:,'Date'])

  outlier_indices = hampel.hampel(df, window_size=window_size, n=n)
  outliers = df.iloc[outlier_indices]

  fig = go.Figure()

  fig.add_trace(go.Scatter(
    x=df.index,
    y=df,
    mode='lines',
    name= 'Normal',
    hovertext = df,   
  ))

  fig.add_trace(go.Scatter(
    x=outliers.index,
    y=outliers,
    mode='markers',
    name= 'Anomaly',
    # hovertext = anomalies2.index,   
  ))

  fig.update_xaxes(title_text = 'Date')
  # fig.update_yaxes(title_text = parameter_of_interest)

  fig.update_layout(
    height=800,
    width=1100,
  )

  fig.show()

In [19]:
hampel_filter('Little Meadows (584)', 'Precipitation Accumulation (in)')

In [28]:
df

Date
1990-10-01     0.0
1990-10-02     0.1
1990-10-03     0.3
1990-10-04     1.1
1990-10-05     1.8
              ... 
2020-09-26    93.4
2020-09-27    93.5
2020-09-28    93.5
2020-09-29    93.5
2020-09-30    93.5
Name: Precipitation Accumulation (in), Length: 10958, dtype: float64

### Local Outlier Factor

In [241]:
def LocalOutlierFactor(station, parameter, plot_type='interactive', contamination=.05, leaf_size=30, seasonal_differencing=True):
  global X, df, predictions, observed
  df = pd.DataFrame(ORWA_Sites2.loc[:, (f'{station}')].loc[:, (f'{parameter}')])  

  if seasonal_differencing == True:    
    #Seasonally decompose df
    # sd = seasonal_decompose(df, freq=365, extrapolate_trend='freq') #https://stackoverflow.com/questions/52422924/stl-decomposition-getting-rid-of-nan-values
    X = bkfilter(df, low=6, high=32, K=12)  #Can change seasonal decomposition to use bk filter. bk filter is a more supported and less naive approach to decomposing season and trend.

    #Grab residuals(data w/o season or trend) and convert to array
    # X = pd.DataFrame(sd.resid) #.dropna()
    X_arr = np.array(X).reshape(-1,1)
    #Scale Data
    scaler = StandardScaler()
    X_arr_scaled = scaler.fit_transform(X_arr)
    #Create dataframe from scaled array
    X_arr_scaled = pd.DataFrame(X_arr_scaled)
    
    #Create model and fit it to scaled data
    lof = LOF(algorithm='auto', contamination=contamination, leaf_size=leaf_size, metric='minkowski',
      metric_params=None, n_jobs=1, n_neighbors=20, p=2)
    lof.fit(X_arr_scaled)
    # LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
    #   metric_params=None, n_jobs=1, n_neighbors=20, p=2) 
    predictions = lof.predict(X_arr_scaled)


    #Combine anomalies with original unscaled residuals
    X['anomaly'] = predictions.tolist()
    #Combine anomalies with the original data
    # observed = sd.observed.dropna()
    df2  = pd.merge(df[f'{parameter}'], X['anomaly'], left_index=True,right_index=True)
    #Filter anomalies from the residual and original datasets
    anomalies_resid_df = X[X['anomaly'] == 1] #Can use if plotting anomalies on residual timeseries
    anomalies2 = df2[df2['anomaly'] == 1]
  
  elif seasonal_differencing == False:

    X_arr = np.array(df).reshape(-1,1)
    #Scale Data
    scaler = StandardScaler()
    X_arr_scaled = scaler.fit_transform(X_arr)
    #Create dataframe from scaled array
    X_arr_scaled = pd.DataFrame(X_arr_scaled)

    lof = LOF(algorithm='auto', contamination=contamination, leaf_size=leaf_size, metric='minkowski',
      metric_params=None, n_jobs=1, n_neighbors=20, p=2)
    lof.fit(X_arr_scaled)
    # LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
    #   metric_params=None, n_jobs=1, n_neighbors=20, p=2)
    df2 = df
    predictions = lof.predict(X_arr_scaled)
    df2['anomaly2'] = predictions
    anomalies2 = df2[df2['anomaly2']==1]

  # matplotlib visualization
  if plot_type == 'static':
    fig, ax = plt.subplots(figsize=(20,12))
    ax.plot(df2.index, df2.iloc[:,0], color='blue', label = 'Normal')
    # ax.plot(X.index, X.iloc[:,0], color='blue', label = 'Normal')
    ax.scatter(pd.to_datetime(anomalies2.index), anomalies2.iloc[:,0], color='red', label = 'Anomaly')
    plt.legend()
    plt.show();
  
# plotly visualization
  elif plot_type == 'interactive':

    fig = go.Figure()

    fig.add_trace(go.Scatter(
      x=df2.index,
      y=df2.iloc[:,0],
      mode='lines',
      name= 'Original Data',
      hovertext = df2.iloc[:,0],   
    ))

    fig.add_trace(go.Scatter(
      x=anomalies2.index,
      y=anomalies2.iloc[:,0],
      mode='markers',
      name= 'Anomaly',
      # hovertext = anomalies2.index,   
    ))

#------------------
    fig.add_trace(go.Scatter(
      x=X.index,
      y=X.iloc[:,0],
      mode='lines',
      name= 'Residuals',
      hovertext = X.iloc[:,0],   
    ))

    fig.add_trace(go.Scatter(
      x=anomalies_resid_df.index,
      y=anomalies_resid_df.iloc[:,0],
      mode='markers',
      name= 'Anomaly',
      # hovertext = anomalies2.index,   
    ))

#----------------
    fig.update_xaxes(title_text = 'Date')
    # fig.update_yaxes(title_text = parameter_of_interest)

    fig.update_layout(
      height=800,
      width=1000,
    )
    fig.show()
  else:
    print('Choose either static or interactive for plot_type')

In [242]:
LocalOutlierFactor('Little Meadows (584)', 'Water Equivalent (in)', plot_type='interactive', seasonal_differencing = True, contamination=0.02, leaf_size=5)

### Isolation Forests

In [180]:
def isolationForest(station, parameter, plot_type, contamination, seasonal_differencing=True, bootstrap=True):

  df = pd.DataFrame(ORWA_Sites2.loc[:, (f'{station}')].loc[:, (f'{parameter}')])

  if seasonal_differencing == True:
    
    #Seasonally decompose df
    # sd = seasonal_decompose(df, freq=365, extrapolate_trend='freq') #https://stackoverflow.com/questions/52422924/stl-decomposition-getting-rid-of-nan-values
    X = bkfilter(df, low=6, high=32, K=12)  #Can change seasonal decomposition to use bk filter. bk filter is a more supported and less naive approach to decomposing season and trend.

    #Grab residuals(data w/o season or trend), convert to array, scale data, convert back to DataFrame
    # X = pd.DataFrame(sd)
    # X = sd.resid
    X_arr = np.array(X)
    scaler = StandardScaler()
    X_arr_scaled = scaler.fit_transform(X)
    X_arr_scaled = pd.DataFrame(X_arr_scaled)

    # train isolation forest
    model =  IsolationForest(n_estimators = 100, contamination = contamination, bootstrap=bootstrap)
    model.fit(X_arr_scaled)
    predictions = model.predict(X_arr_scaled)
    
    X['anomaly'] = predictions.tolist()
    observed = sd.observed.dropna()
    df2  = pd.merge(observed, X['anomaly'], left_index=True,right_index=True)

    anomalies = X[X['anomaly'] == -1] #Can use if plotting anomalies on residual timeseries
    anomalies2 = df2[df2['anomaly'] == -1]

  elif seasonal_differencing == False:
    X = np.array(df.dropna())
    scaler = StandardScaler()
    np_scaled = scaler.fit_transform(X)
    X = pd.DataFrame(np_scaled)
    # train isolation forest
    model =  IsolationForest(n_estimators = 100, contamination = 0.1, bootstrap=bootstrap)
    model.fit(X)

    df2 = df #Keep original data untouched
    df2['anomaly2'] = model.predict(X)
    anomalies2 = df2.loc[df2['anomaly2'] == -1]

  # matplotlib visualization
  if plot_type == 'static':
    fig, ax = plt.subplots(figsize=(20,12))
    ax.plot(df2.index, df2.iloc[:,0], color='blue', label = 'Normal')
    # ax.plot(X.index, X.iloc[:,0], color='blue', label = 'Normal')
    ax.scatter(pd.to_datetime(anomalies2.index), anomalies2.iloc[:,0], color='red', label = 'Anomaly')
    plt.legend()
    plt.show();

  # plotly visualization
  elif plot_type == 'interactive':

    fig = go.Figure()

    fig.add_trace(go.Scatter(
      x=df2.index,
      y=df2.iloc[:,0],
      mode='lines',
      name= 'Normal',
      hovertext = df2.iloc[:,0],   
    ))

    fig.add_trace(go.Scatter(
      x=anomalies2.index,
      y=anomalies2.iloc[:,0],
      mode='markers',
      name= 'Anomaly',
      # hovertext = anomalies2.index,   
    ))

    fig.update_xaxes(title_text = 'Date')
    # fig.update_yaxes(title_text = parameter_of_interest)

    fig.update_layout(
      height=800,
      width=1000,
    )
    fig.show()
  else:
    print('Choose either static or interactive for plot_type')

In [236]:
# isolationForest('Little Meadows (584)', 'Water Equivalent (in)','interactive', .01, True, True)

### Seasonal Decomposition

In [100]:
def seasonal_decomposition(station, parameter, method = 'STL'):
  # global sd, bk, X, df2, anomalies, stl
  df = ORWA_Sites2.loc[:, (f'{station}')].loc[:, (f'{parameter}')] 

  # if method == 'naive':
  #   sd = seasonal_decompose(df, model = 'additive', period=365, extrapolate_trend='freq')  #https://stackoverflow.com/questions/52422924/stl-decomposition-getting-rid-of-nan-values
  #   X = sd.resid.dropna()
  #   obs = sd.observed.dropna()
  #   # df2  = pd.merge(observed, X['zscore'], left_index=True,right_index=True)
  #   # X.plot()

  # elif method == 'BaxterKing':
  #   bk = bkfilter(df, low=6, high=32, K=12)  #Can change seasonal decomposition to use bk filter. bk filter is a more supported and less naive approach to decomposing season and trend.   
  #   X = bk.dropna().iloc[:,0]
  #   # df2 = X
    # X.plot()

  if method == 'STL':
    stl = STL(df, robust = True, seasonal = 13)
    res = stl.fit()
    # X = res
    res.plot()
    # res.plot(observed=False, resid=True, seasonal=True, trend=True)

  elif method == 'naive' or 'BaxterKing':
    if method == 'naive':
      sd = seasonal_decompose(df, model = 'additive', period=365, extrapolate_trend='freq')  #https://stackoverflow.com/questions/52422924/stl-decomposition-getting-rid-of-nan-values
      X = sd.resid.dropna()
      obs = sd.observed.dropna()
      # df2  = pd.merge(observed, X['zscore'], left_index=True,right_index=True)
      # X.plot()

    elif method == 'BaxterKing':
      bk = bkfilter(df, low=6, high=32, K=12)  #Can change seasonal decomposition to use bk filter. bk filter is a more supported and less naive approach to decomposing season and trend.   
      X = bk.dropna()
      # df2 = X
      # X.plot()   

    fig = go.Figure()

    fig.add_trace(go.Scatter(
      x=X.index,
      y=X.iloc[:],
      mode='lines',
      name= 'Normal',
      hovertext = X.iloc[:],   
    ))

    fig.update_xaxes(title_text = 'Date')

    fig.update_layout(
      height=600,
      width=1300,
    )
    fig.show()



In [142]:
# seasonal_decomposition('Little Meadows (584)', 'Water Equivalent (in)', method='STL')

In [141]:
# seasonal_decomposition('Little Meadows (584)', 'Water Equivalent (in)', method='BaxterKing')

### Z Score

In [117]:
def zScore_plot(station, parameter, max_StDev=2.5, plot_type = 'interactive', seasonal_differencing = True):
  global zs, X, df
  df = ORWA_Sites2.loc[:, (f'{station}')].loc[:, (f'{parameter}')] 

  if seasonal_differencing == True:
    #Remove season and trend.
    sd = seasonal_decompose(df,period=365, extrapolate_trend='freq')  #https://stackoverflow.com/questions/52422924/stl-decomposition-getting-rid-of-nan-values
    # X = bkfilter(df, low=6, high=32, K=12)  #Can change seasonal decomposition to use bk filter. bk filter is a more supported and less naive approach to decomposing season and trend.   
    X = pd.DataFrame(sd.resid.dropna())
    zs = zscore(X, ddof=1, nan_policy='omit')
    X['zscore'] = zs

    observed = sd.observed.dropna()
    df2  = pd.merge(observed, X['zscore'], left_index=True,right_index=True)
    anomalies = df2[(df2['zscore'] < -max_StDev) | (df2['zscore'] > max_StDev)]

  elif seasonal_differencing == False:
    # X_arr = np.array(X)
    X = df.dropna()
    zs = zscore(X.iloc[:,0], ddof=2, nan_policy='omit')
    X['zscore'] = zs

    df2 = X
    anomalies = X[(X['zscore'] < -max_StDev) | (X['zscore'] > max_StDev)]

  # matplotlib visualization
  if plot_type == 'static':

    fig, ax = plt.subplots(figsize=(20,12))
    ax.plot(df2.index, df2.iloc[:,0], color='blue', label = 'Normal')
    ax.scatter(pd.to_datetime(anomalies.index), anomalies.iloc[:,0], color='red', label = 'Anomaly')
    plt.legend()
    plt.show();

  # plotly visualization
  elif plot_type == 'interactive':

    fig = go.Figure()

    fig.add_trace(go.Scatter(
      x=df2.index,
      y=df2.iloc[:,0],
      mode='lines',
      name= 'Normal',
      hovertext = df2.iloc[:,0],   
    ))

    fig.add_trace(go.Scatter(
      x=anomalies.index,
      y=anomalies.iloc[:,0],
      mode='markers',
      name= 'Anomaly',
      hovertext = anomalies.index,   
    ))

    fig.update_xaxes(title_text = 'Date')
    # fig.update_yaxes(title_text = parameter_of_interest)

    fig.update_layout(
      height=800,
      width=1000,
    )
    fig.show()
  else:
    'Choose either static or interactive for plot_type'

In [140]:
# zScore_plot('Little Meadows (584)', 'Water Equivalent (in)')

### Double Mass

In [125]:
def double_mass(stations, parameter, month=0):  

 #Compile and filter data with respect to inputs:
  site_list = [ORWA_Sites2[i][parameter] for i in stations] 
  df_merged = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), site_list).dropna()
  df2 = df_merged
  df2.columns = [f'{i} {parameter}' for i in stations]
  
  if month == 0:
    df2 = df2
    # print(df2) 
  elif month > 0 or month < 12:
    df2 = df2[df2.index.month == month] #need to understand indexes and datatime indices better.  When to use .dt accessor?

#Double Mass Analysis
  x=pd.DataFrame(abs(df2.iloc[:,0].diff()).cumsum())
  x.columns = ['x']
  y=pd.DataFrame(np.add.reduce([abs(df2.iloc[:,i].diff()).cumsum() for i in range(len(stations))]))  #might want to try to use pd.apply(np.cumsum) here to reduce to maintain datetime index
  y.index = x.index
  y.columns= ['y']

#Plot the data
  customdata_list = [df2.iloc[:,0], x, y]
  customdata = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), customdata_list)
  customdata.reset_index(inplace=True)

  fig = make_subplots(
      rows=1, cols=1,
      subplot_titles=(f'{parameter} Double Mass Curve', f'Yearly {parameter} Double Mass Curve')
  )

  fig.add_trace(go.Scatter(
      x=x.iloc[:,0],
      y=y.iloc[:,0],
      mode='lines',
      customdata=customdata,
      hovertemplate = '<b>Date: %{customdata[0]}</b>', #<br>Site of Interest - Cumulative {parameter}: %{customdata[2]:.2f}</br>Comparison Site(s) - Cumulative SWE (in): %{customdata[3]:.2f}<br>Site of Interest - SWE (in): %{customdata[1]:.2f}</br>',
      # hovertemplate = f'<b>Date: %{customdata.iloc[:,0]}</b><br>Site of Interest - Cumulative:  %{customdata.iloc[:,2]}</br>', #Comparison Site(s) - Cumulative {parameter}: {customdata[3]}<br>Site of Interest - Daily {parameter}: {customdata[1]}</br>',
      name = 'Double mass curve'
  ), 
  row=1, col=1)

  fig.update_xaxes(title_text= f"Cumulative {parameter} at {stations[0]}", row=1, col=1)
  fig.update_yaxes(title_text=f"Total Cumulative {parameter} of Comparison Sites <br>{stations[1::]}</br>", row=1, col=1)

  fig.update_layout(
   height=800,
   width=1200
  )
  
  fig.show()

In [139]:
# double_mass(['Little Meadows (584)', 'Aneroid Lake #2 (302)'], 'Water Equivalent (in)')

### Box and Whisker Plots

In [136]:
def boxplots(station, parameter, data_pt_type):
  """
  options for data_pt_type include: 'all', 'outliers', or 'suspectedoutliers'
  """      
  df = pd.DataFrame(ORWA_Sites2.loc[:, (f'{station}')].loc[:, (f'{parameter}')])

  fig = go.Figure()
  for i in df.columns:
  #   WA = WA_Sites[WA_Sites[i] != 0].dropna()

    fig.add_trace(go.Box(y = df.loc[:,i], 
            name = i,
            boxpoints=data_pt_type, 
            # hovertext = str(df_temp[i].index),
            jitter=0.2,
            whiskerwidth=0.5                
  ))

  fig.update_layout(
          autosize=False,
          width=1200,
          height=500,
          margin=dict(l=20, r=20, t=2, b=8),
          paper_bgcolor="LightSteelBlue",
          title={
          # 'text': f'{station}',
          'y':0.98,
          'x':0.45,
          'xanchor': 'center',
          'yanchor': 'top'
  })
  fig.show()

In [138]:
# boxplots('Little Meadows (584)', 'Water Equivalent (in)', data_pt_type='outliers')

# Analysis

In [ ]:
SWEvsPrecip('Little Meadows (584)')

In [ ]:
SWEvsPrecip('Lyman Lake (606)')

### Reporting of errors

In [ ]:
def SWEvsPrecip_table(station):  

  global df, df2, df_errors
#Create dataframe with WTEQ and Precip Incr for selected station
  df = ORWA_Sites2.loc[:, (f'{station}')].loc[:, ('Water Equivalent (in)', 'Precipitation Accumulation (in)')]  
  df.reset_index(inplace=True)
  pd.to_datetime(df.loc[:,'Date'])

#Mask to select only data between Nov - Apr
  winter_filter = ~df.loc[:,'Date'].dt.month.between(5,10)
  df2 = df.where(winter_filter).dropna()
  # df2.set_index('Date', inplace = True)

#Add column specifying the water year (Oct 1 - Sep 30) of each record
  df2.reset_index(inplace=True)  #Have to reset the index so the dates become just a regular datetime object to get .dt.year to work.
  pd.to_datetime(df2.loc[:,'Date'])
  df2.loc[:,'water_year'] = df2.loc[:,'Date'].dt.year.where(df2.loc[:,'Date'].dt.month < 10, df2.loc[:,'Date'].dt.year + 1)
  df2.loc[:,'water_year'] = list(map(lambda x: str(x), df2.loc[:,'water_year']))

#Calculate Precip increment from accumulation:
  df2['Precip Increment - Calculated (in)'] = df2['Precipitation Accumulation (in)'].diff()


#Filter to retrieve potentially erroneous data
  df2['SWE Increment (in)'] = df2['Water Equivalent (in)'].diff()
  df_errors = df2[(df2['Precip Increment - Calculated (in)'] <= 0.2) & (df2['SWE Increment (in)'] >= 0.8)].sort_values('SWE Increment (in)', ascending=False)
  # df_errors = df2[(df2['Precipitation Increment (in)'] <= 0.1) & (df2['SWE Increment (in)'] >= 1)].sort_values('SWE Increment (in)', ascending=False)  #Need a SWE increment
  df_errors.to_excel('./SWEvsPrec/'+f'{station}' + '_SWEvsPrecipErrors.xlsx')
  df_errors.set_index('Date', inplace=True)
  df_errors.drop('index', axis=1, inplace=True)

  return df_errors

In [ ]:
for i in stations:
  SWEvsPrecip(i)

In [ ]:
stations

NameError: ignored